In [1]:
import numpy as np              
import pandas as pd
import os  

from multiprocessing import Pool

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Filtrage
dfs = []
fields = ["GlobalEventID",  "Day", "MonthYear", "Year",   "FractionDate",   "Actor1Code", "Actor1Name",
                "Actor1CountryCode", "Actor1KnownGroupCode",  "Actor1EthnicCode", "Actor1Religion1Code",
                "Actor1Religion2Code", "Actor1Type1Code", "Actor1Type2Code", "Actor1Type3Code", "Actor2Code",
                "Actor2Name", "Actor2CountryCode", "Actor2KnownGroupCode", "Actor2EthnicCode", "Actor2Religion1Code",
                "Actor2Religion2Code", "Actor2Type1Code", "Actor2Type2Code", "Actor2Type3Code", "IsRootEvent",
                "EventCode", "EventBaseCode", "EventRootCode", "QuadClass", "GoldsteinScale", "NumMentions",
                "NumSources", "NumArticles", "AvgTone", "Actor1Geo_Type", "Actor1Geo_Fullname",
                "Actor1Geo_CountryCode", "Actor1Geo_ADM1Code", "Actor1Geo_ADM2Code", "Actor1Geo_Lat", "Actor1Geo_Long",
                "Actor1Geo_FeatureID", "Actor2Geo_Type", "Actor2Geo_Fullname", "Actor2Geo_CountryCode",
                "Actor2Geo_ADM1Code", "Actor2Geo_ADM2Code", "Actor2Geo_Lat", "Actor2Geo_Long", "Actor2Geo_FeatureID",
                "ActionGeo_Type", "ActionGeo_Fullname", "ActionGeo_CountryCode", "ActionGeo_ADM1Code",
                "ActionGeo_ADM2Code", "Action2Geo_Lat", "Action2Geo_Long", "Action2Geo_FeatureID", "DATEADDED",
                "SOURCEURL"]

In [3]:
# april
for day in ['21','22','23','24','25','26','27','28','29','30']:
    for hours in ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']:
        for minutes in ['00','15','30','45']:
            code = '201704' + day + hours + minutes + '00'
            if (os.path.isfile('./data/' +  code + '.export.CSV')):
                df = pd.read_csv('./data/' +  code + '.export.CSV', sep='\t', header=None, names=fields)
                df = df[['GlobalEventID','Day', 'Actor1CountryCode', 'Actor2CountryCode', 'GoldsteinScale', 'NumMentions', 'NumSources', 
                'NumArticles', 'AvgTone',  'Actor1Geo_Lat','Actor1Geo_Long', 'Actor2Geo_Lat', 'Actor2Geo_Long', 'DATEADDED', 'SOURCEURL']]
                df = df[(df['Actor1CountryCode'].notnull()) & (df['Actor2CountryCode'].notnull())]
                df['DATEADDED'] = df['DATEADDED'].replace(code, int('201704' + day))
                if (df.shape[0] != 0):
                    dfs.append(df)

In [4]:
# may 
for day in ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21']:
    for hours in ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']:
        for minutes in ['00','15','30','45']:
            code = '201705' + day + hours + minutes + '00'
            if (os.path.isfile('./data/' +  code + '.export.CSV')):
                df = pd.read_csv('./data/' +  code + '.export.CSV', sep='\t', header=None, names=fields)
                df = df[['GlobalEventID','Day', 'Actor1CountryCode', 'Actor2CountryCode', 'GoldsteinScale', 'NumMentions', 'NumSources', 
                'NumArticles', 'AvgTone',  'Actor1Geo_Lat','Actor1Geo_Long', 'Actor2Geo_Lat', 'Actor2Geo_Long', 'DATEADDED', 'SOURCEURL']]
                df = df[(df['Actor1CountryCode'].notnull()) & (df['Actor2CountryCode'].notnull())]
                df['DATEADDED'] = df['DATEADDED'].replace(code, int('201705' + day))
                if (df.shape[0] != 0):
                    dfs.append(df)

In [8]:
from pymongo import MongoClient

client = MongoClient('ec2-54-173-229-9.compute-1.amazonaws.com:27026')
db = client.gdelt
events = db.events

In [9]:
def loadMongo(df):
    data = df.T.to_dict().values()
    result = events.insert_many(data)
    return 0

In [10]:
import time 
t0 = time.time()
p = Pool(5)
p.map(loadMongo,dfs)
t1 = time.time()
print("TIME : ", t1-t0)

TIME :  102.5484471321106
